In [5]:
import pandas as pd

df = pd.read_csv("data/dataset.csv")

In [9]:
labels = {
    # Valence
    "happy": [0.7, 1.0],
    "neutral_mood": [0.3, 0.7],
    "sad": [0.0, 0.3],

    # Popularity
    "high_popularity": [0.7, 1.0],
    "mid_popularity": [0.3, 0.7],
    "low_popularity": [0.0, 0.3],

    # Duration (normalized to [0.0, 1.0])
    "long_duration": [0.7, 1.0],
    "medium_duration": [0.3, 0.7],
    "short_duration": [0.0, 0.3],

    # Danceability
    "high_danceability": [0.7, 1.0],
    "mid_danceability": [0.3, 0.7],
    "low_danceability": [0.0, 0.3],

    # Energy
    "high_energy": [0.7, 1.0],
    "mid_energy": [0.3, 0.7],
    "low_energy": [0.0, 0.3],

    # Loudness (normalized to [0.0, 1.0])
    "high_loudness": [0.7, 1.0],
    "mid_loudness": [0.3, 0.7],
    "low_loudness": [0.0, 0.3],

    # Speechiness
    "high_speechiness": [0.7, 1.0],
    "mid_speechiness": [0.3, 0.7],
    "low_speechiness": [0.0, 0.3],

    # Acousticness
    "high_acousticness": [0.7, 1.0],
    "mid_acousticness": [0.3, 0.7],
    "low_acousticness": [0.0, 0.3],

    # Instrumentalness
    "high_instrumentalness": [0.7, 1.0],
    "mid_instrumentalness": [0.3, 0.7],
    "low_instrumentalness": [0.0, 0.3],

    # Liveness
    "high_liveness": [0.7, 1.0],
    "mid_liveness": [0.3, 0.7],
    "low_liveness": [0.0, 0.3],

    # Tempo (normalized to [0.0, 1.0])
    "high_tempo": [0.7, 1.0],
    "mid_tempo": [0.3, 0.7],
    "low_tempo": [0.0, 0.3],
}

In [13]:
# Define min and max values for each feature (adjust based on your dataset)
feature_ranges = {
    "valence": (0.0, 1.0),
    "popularity": (0, 100),
    "duration_ms": (0, 600000),  # Adjust based on your dataset
    "danceability": (0.0, 1.0),
    "energy": (0.0, 1.0),
    "loudness": (-60, 0),  # Loudness ranges from -60 dB to 0 dB
    "speechiness": (0.0, 1.0),
    "acousticness": (0.0, 1.0),
    "instrumentalness": (0.0, 1.0),
    "liveness": (0.0, 1.0),
    "tempo": (0, 250),  # Tempo ranges from 0 BPM to 250 BPM
}

# Normalize features to [0.0, 1.0]
for feature, (min_val, max_val) in feature_ranges.items():
    df[f"{feature}_normalized"] = (df[feature] - min_val) / (max_val - min_val)

# Display the updated dataset
print(df.head())

   Unnamed: 0                track_id                 artists  \
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   
